# Natural Language Processing

Today, we are going to explore using text as data. We will cover the following topics (each one very briefly):

1. Preprocessing text (i.e. cleaning text)
2. Topic modeling with LDA
3. Word vectors using word2vec

To get started, please install `gensim`. You can do so using:

```
conda install -c anaconda gensim
```


## Getting Started

Let's import the usual suspects: `pandas`, `matplotlib`, `numpy`, and our new favorite library, `gensim`.

Gensim is a very powerful module for performing all sorts of natural language processing. It has become the default for word embedding (word vector) models like word2vec and doc2vec. Because `gensim` is very large, we won't import the whole thing. We'll only import the parts that we're going to need.

For many problems, you may want to refer to the Gensim documentation. This page will be particularly helpful: https://radimrehurek.com/gensim/models/ldamodel.html

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## For preprocessing
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import strip_short

## For topic modeling
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel

## For word embedding models
from gensim.models import Word2Vec


## Problem 1

Use Pandas to read our `twitter.csv` dataset. Name the new dataframe `docs`. Use `head()` to inspect the data.

In [32]:
docs = pd.read_csv("twitter.csv")
print(docs.info())
docs.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5113 entries, 0 to 5112
Data columns (total 5 columns):
Topic        5113 non-null object
Sentiment    5113 non-null object
TweetId      5113 non-null int64
TweetDate    5113 non-null object
TweetText    5113 non-null object
dtypes: int64(1), object(4)
memory usage: 199.9+ KB
None


,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...


## Problem 2

Wow. That's some messy text. What happens if we use the gensim preprocessing functions we imported above to clean it up a little bit? Go ahead and try it. Make a list called `clean_tweets` and perform the following operations on `docs["TweetText"]`, storing the results each time in `clean_tweets`.

Print the first three elements of `clean_tweets` each time to see how things are changing.

### Hint

The Gensim functions expect they will only get one string (i.e. tweet) at a time. Therefore, you must use list comprehensions to process your list of `clean_tweets`. For example: 
```python
clean_tweets = [function(tweet) for tweet in clean_tweets]
```


### Problem 2a
Strip all punctuation. 


In [33]:
from gensim.parsing.preprocessing import strip_punctuation
clean_tweets = [strip_punctuation(tweet) for tweet in docs.TweetText]
for i in range(3):
    print(str(i+1) + ")", clean_tweets[i])


1) Now all  Apple has to do is get swype on the iphone and it will be crack  Iphone that is
2)  Apple will be adding more carrier support to the iPhone 4S  just announced 
3) Hilarious  youtube video   guy does a duet with  apple  s Siri  Pretty much sums up the love affair  http t co 8ExbnQjY


### Problem 2b

Strip multiple whitespaces.

In [34]:
from gensim.parsing.preprocessing import strip_multiple_whitespaces
clean_tweets = [strip_multiple_whitespaces(tweet) for tweet in clean_tweets]
for i in range(3):
    print(str(i+1) + ")", clean_tweets[i])

1) Now all Apple has to do is get swype on the iphone and it will be crack Iphone that is
2)  Apple will be adding more carrier support to the iPhone 4S just announced 
3) Hilarious youtube video guy does a duet with apple s Siri Pretty much sums up the love affair http t co 8ExbnQjY


### Problem 2c

Strip numeric values from the strings.

In [35]:
from gensim.parsing.preprocessing import strip_numeric
clean_tweets = [strip_numeric(tweet) for tweet in clean_tweets]
for i in range(3):
    print(str(i+1) + ")", clean_tweets[i])

1) Now all Apple has to do is get swype on the iphone and it will be crack Iphone that is
2)  Apple will be adding more carrier support to the iPhone S just announced 
3) Hilarious youtube video guy does a duet with apple s Siri Pretty much sums up the love affair http t co ExbnQjY


### Problem 2d

Strip all of the English stopwords from the tweets. A stopword is a common word that does not add value to our data. For example: "The", "a", "were", "are", "be", "is", "there" are all stopwords.

In [36]:
from gensim.parsing.preprocessing import remove_stopwords
clean_tweets = [remove_stopwords(tweet) for tweet in clean_tweets]
for i in range(3):
    print(str(i+1) + ")", clean_tweets[i])


1) Now Apple swype iphone crack Iphone
2) Apple adding carrier support iPhone S announced
3) Hilarious youtube video guy duet apple s Siri Pretty sums love affair http t ExbnQjY


### Problem 2e

Finally, let's drop all of the words less than three characters in length. Use `strip_short` to do this.

In [37]:
from gensim.parsing.preprocessing import strip_short
clean_tweets = [strip_short(tweet, minsize=3) for tweet in clean_tweets]
for i in range(3):
    print(str(i+1) + ")", clean_tweets[i])

1) Now Apple swype iphone crack Iphone
2) Apple adding carrier support iPhone announced
3) Hilarious youtube video guy duet apple Siri Pretty sums love affair http ExbnQjY


## Problem 3

Many functions in Gensim expect the corpus (collection of texts) to be a lists of lists. Our corpus is our list of tweets. However, right now that is simply a list full of strings. Each string is a tweet (or "document"). We need to transform our corpus into a list of lists. The outer list contains all of the documents. The inner lists each represent a document. They contain strings. Each string is an individual word. For example:

```
[
  ["docA_word1","docA_word2","docA_word3"],
  ["docB_word1","docB_word2","docB_word3","docB_word4"],
  ...,
  ["docZ_word1","docZ_word2"]
]
```

Again, use a list comprehension to split every document. You can use the built-in `split()` method to split a single string into a list of words. The default split behavior is to divide the sentence up based on whitespace. Call your new corpus `tokenized_docs`. 

Tokenization is the process of splitting a string (or document or text) into a collection of tokens (typically words).

Print the first three elements of your `tokenized_docs` object.

In [38]:
clean_tweets = [str.split(tweet) for tweet in clean_tweets]
for i in range(3):
    print(str(i+1) + ")", clean_tweets[i])

1) ['Now', 'Apple', 'swype', 'iphone', 'crack', 'Iphone']
2) ['Apple', 'adding', 'carrier', 'support', 'iPhone', 'announced']
3) ['Hilarious', 'youtube', 'video', 'guy', 'duet', 'apple', 'Siri', 'Pretty', 'sums', 'love', 'affair', 'http', 'ExbnQjY']


## Problem 4

Once we have our corpus represented as a list of lists and our documents are all tokenized, we can use Gensim's `Dictionary` function to make a dictionary that represents the vocabulary of our corpus. Do that below. Call your dictionary `dictionary`. 

If you then `print(dictionary)`, it will show you some of your terms and also tell you how many unique tokens are in your dataset.

In [45]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(clean_tweets)

print(dictionary)

Dictionary(16840 unique tokens: ['Apple', 'Iphone', 'Now', 'crack', 'iphone']...)


## Problem 5

Now we need to convert each document into a new data structure called a bag of words. A bag of words representation of a document is a vector the same length as the number of words in your vocabulary (or dictionary). If you have 10,000 unique words, every document will be represented by a length 10,000 vector. This vector is 0 for all words that _do not_ appear in a document. For every word that _does_ appear in that document, the corresponding entry in the bag of words vector is the _count_ of the number of times that word appears in the document.

Use the `doc2bow` method of your `dictionary` to convert every document in your corpus into a bag of words.

(Again, see here: https://radimrehurek.com/gensim/models/ldamodel.html)

Call your new list `tweets_bow`. Because Gensim uses an efficient method to store the bag of words data, printing this object will not be very useful or insightful.

In [47]:
from gensim.test.utils import common_texts

# Create a corpus from a list of texts
common_dictionary = Dictionary(common_texts)
tweets_bow = [dictionary.doc2bow(text) for text in common_texts]
tweets_bow

[[(7607, 1), (9605, 1)],
 [(41, 1), (298, 1), (994, 1), (3288, 1)],
 [(41, 1), (7607, 1)],
 [(9605, 1)],
 [(41, 1), (298, 1), (3288, 1)],
 [],
 [],
 [],
 [(994, 1)]]

## Problem 6

Now, use `gensim`'s `LdaModel` to estimate a topic model of our tweets with `num_topics=4`. Call your model `lda`. 

Use the following hyperparameters or arguments:

* `num_topics=4`
* `id2word=dictionary`
* `passes=5`

## Problem 7

Use the `print_topics` method to print the top terms for each of your topics. 

## Problem 8

Let's make predictions for which topic each of our Tweets is in. Use the `get_document_topics()` method to get the topic distribution for all of our tweets.

1. First, give `get_document_topics()` your `tweets_bow` object. Call the output of this function `topics`.
2. This returns a complicated list of list of tuples. It looks like below:

```
  [
    [ (0, Pr(t=0)), (1, Pr(t=1)),..., (3, Pr(t=3)) ],
    ...
    [ (0, Pr(t=0)), (1, Pr(t=1)),..., (3, Pr(t=3)) ]
  ]
```
3. Write a loop that iterates over `topics` and finds the highest probability topic for each document. Record the topic number with the highest probability for every tweet in a new list called `best_topics`. You can also do this with a list comprehension.

## Problem 9

We know the "true" topic for every tweet. It is in `docs["Topic"]`. These were hand-labeled. We did not tell our model _anything_ about these labels. Let's see if it was able to discern these four topics.

Make a contingency matrix of `docs["Topic"]` and `best_topics`. Print the contingency matrix. You do not need to plot it visually. The easiest way to do this is going to be the following:

1. Make a new dictionary that contains keys "actual" and "pred".
2. The value of "actual" should be `docs["Topic"].tolist()`.
3. The value of "pred" should be `best_topics`.
4. Use the pandas `DataFrame` method to turn your dictionary into a pandas dataframe.
5. Now, use the `pd.crosstab` method with `margins=False` to print your contingency matrix.

## Problem 10

Now, let's learn a word vector model using word2vec. Word vectors are real-valued representations of words that retain the words' meanings. Words with similar meanings will have similar word vectors. For example, `dog` will be closer to `wolf` than it is to `house`. We can also perform algebra on word vectors. The common example is:

$$
vector(king) + vector(woman) - vector(man) \approx vector(queen)
$$

The first step is to read in a somewhat larger and more interesting dataset on hotel reviews. Use pandas' to load `hotel_reviews.csv` into a dataframe called `hotels`. Print the head of `hotels`. 

## Problem 11

Next, preprocess the text of each review using `gensim` following the same steps as in Problem 2 and Problem 3. You should finish with a list of tokenized documents (each document is a hotel review). You can call this `tokenized_reviews`.  

This time, also convert all of the texts to lowercase. You can accomplish this by using the `.lower()` method as shown below:

In [ ]:
"This is AN eXaMPLe".lower()

## Problem 12

Use gensim's `Word2Vec` to train a word2vec model called `w2v`. Set the following arguments:

* `size=100`
* `min_count=5`

## Problem 13

Use the `most_similar` method to find the words that are most similar to "clean". Also use the function to find the words that are most similar to "dirty".

## Problem 14

Use the `most_similar` method to find words that satisfy the following equation:

$$
best + bad - good = ?
$$

This correspond to the anaology:

```
good:best::bad:?
```

You can use the `positive` and `negative` arguments of `most_similar` to construct your query.